In [3]:
import pandas as pd
import os

# Load dateset
train_df = pd.read_csv(os.getcwd() + '/../../data/bot-iot/Bot-IoT_Dataset/Dataset/5%/10-best features/10-best Training-Testing split/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv', low_memory=False)
attack_df_train = train_df[train_df['attack'] == 1]
attack_df_train = attack_df_train.drop(columns=['attack', 'category', 'subcategory'])
normal_df_train = train_df[train_df['attack'] == 0]
normal_df_train = normal_df_train.drop(columns=['attack', 'category', 'subcategory'])

test_df = pd.read_csv(os.getcwd() + '/../../data/bot-iot/Bot-IoT_Dataset/Dataset/5%/10-best features/10-best Training-Testing split/UNSW_2018_IoT_Botnet_Final_10_best_Testing.csv', low_memory=False)
attack_df_test = test_df[test_df['attack'] == 1]
attack_df_test = attack_df_test.drop(columns=['attack', 'category', 'subcategory'])
normal_df_test = test_df[test_df['attack'] == 0]
normal_df_test = normal_df_test.drop(columns=['attack', 'category', 'subcategory'])

print("Attack Training set size: ", attack_df_train.shape)
print("Attack Test set size: ", attack_df_test.shape)

print("Normal Training set size: ", normal_df_train.shape)
print("Normal Test set size: ", normal_df_test.shape)

Attack Training set size:  (2934447, 16)
Attack Test set size:  (733598, 16)
Normal Training set size:  (370, 16)
Normal Test set size:  (107, 16)


In [5]:
# pip install langchain
# pip install langchain-openai
# pip install langchain_ollama
# pip install langchain-huggingface
# pip install sentence-transformers
# pip install langchain-chroma
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
# https://python.langchain.com/v0.2/docs/integrations/text_embedding/huggingfacehub/

from langchain_openai import OpenAIEmbeddings
from langchain_ollama import OllamaEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from tqdm import tqdm
import os
import dotenv

# Initialize embeddings

# 1. OpenAIEmbeddings
# dotenv.load_dotenv()
# API_KEY = os.getenv("MY_OPENAI_API_KEY")
# embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=API_KEY)

# 2. OllamaEmbeddings
# embeddings = OllamaEmbeddings(model="all-minilm")

# 3. HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

# Initialize vector store
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory="./chroma_db_binary")

# Add batched documents to vector store
batch_size = 16
total_attack_documents = 10 #attack_df_train.shape[0]
total_normal_documents = 10 #normal_df_train.shape[0]

for i in tqdm(range(0, total_attack_documents, batch_size), ncols=100, desc="Adding attack documents to vector store"):
    documents = [Document(page_content=str(attack_df_train.iloc[i+j].to_list()), metadata={"source": "bot-iot", "label": "attack"}) for j in range(min(batch_size, total_attack_documents-i))]
    vector_store.add_documents(documents)

for i in tqdm(range(0, total_normal_documents, batch_size), ncols=100, desc="Adding normal documents to vector store"):
    documents = [Document(page_content=str(normal_df_train.iloc[i+j].to_list()), metadata={"source": "bot-iot", "label": "normal"}) for j in range(min(batch_size, total_normal_documents-i))]
    vector_store.add_documents(documents)

print(f"Total number of documents added: {total_attack_documents+total_normal_documents}")

/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Adding normal documents to vector store: 100%|████████████████████████| 1/1 [00:01<00:00,  1.38s/it]

Total number of documents added: 20
